In [2]:
%load_ext autoreload
%autoreload 2

In [47]:
import httpx
import asyncio
import pandas as pd
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy.ext.asyncio import create_async_engine

In [43]:
from pokemon.typing import Pokemon
from pokemon import db

In [32]:
poke_requests = [api.get_pokemon_by_id(n) for n in range(1,300)]

In [14]:
async def fetch_requests(requests):
    async with httpx.AsyncClient() as client:
        poke_tasks = [client.send(request) for request in requests]
        return await asyncio.gather(*poke_tasks)

In [33]:
poke_responses = await fetch_requests(poke_requests)

In [37]:
pokemons = [Pokemon(**response.json()) for response in poke_responses]
pokemons_orm = [db.Pokemons(**pokemon.dict()) for pokemon in pokemons]

[Pokemon(id=1, name='bulbasaur', height=7, weight=69, base_experience=64),
 Pokemon(id=2, name='ivysaur', height=10, weight=130, base_experience=142),
 Pokemon(id=3, name='venusaur', height=20, weight=1000, base_experience=263),
 Pokemon(id=4, name='charmander', height=6, weight=85, base_experience=62),
 Pokemon(id=5, name='charmeleon', height=11, weight=190, base_experience=142),
 Pokemon(id=6, name='charizard', height=17, weight=905, base_experience=267),
 Pokemon(id=7, name='squirtle', height=5, weight=90, base_experience=63),
 Pokemon(id=8, name='wartortle', height=10, weight=225, base_experience=142),
 Pokemon(id=9, name='blastoise', height=16, weight=855, base_experience=265),
 Pokemon(id=10, name='caterpie', height=3, weight=29, base_experience=39),
 Pokemon(id=11, name='metapod', height=7, weight=99, base_experience=72),
 Pokemon(id=12, name='butterfree', height=11, weight=320, base_experience=198),
 Pokemon(id=13, name='weedle', height=3, weight=32, base_experience=39),
 Pokem

In [46]:
pokemons_orm